In [1]:
import os, openai, json
import keys, modify_spec
os.environ["OPENAI_API_KEY"] = keys.OPEN_AI_KEY
os.environ["HELIUS_API_KEY"] = keys.HELIUS_KEY
openai.api_key = keys.OPEN_AI_KEY

In [2]:
"""
openapi: 3.0.2
info:
  title: Retrieval Plugin API
  description: A retrieval API for querying and filtering documents based on natural language queries and metadata
  version: 1.0.0
servers:
  - url: http://localhost:3000
    description: Localhost server
paths:
  /api/handlers/getAssetsByOwner:
    post:
      summary: getAssetsByOwner
      description: Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address
      operationId: query_assets_by_owner
      requestBody:
        content:
          application/json:
            schema:
              $ref: "#/components/schemas/getAccountInfoRequest"
        required: true
      responses:
        "200":
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: "#/components/schemas/getAssetsByOwnerResponse"
        "500":
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: "#/components/schemas/HTTPValidationError"
      security:
        - HTTPBearer: []
        schemas:
            getAccountInfoRequest:
            title: GetAccountInfoRequest
            type: object
            required:
                - address
            properties:
                address:
                title: Address
                type: string
            getAccountInfoResponse:
            title: GetAccountInfoResponse
            type: object
            properties:
                message:
                title: Message
                type: object
                properties:
                    data:
                    title: Data
                    type: object
                    properties:
                        type:
                        title: Type
                        type: string
                        data:
                        title: Data
                        type: array
                        items: { type: string }
                    executable:
                    title: Executable
                    type: boolean
                    lamports:
                    title: Lamports
                    type: number
                    owner:
                    title: Owner
                    type: string
                    rentEpoch:
                    title: Owner
                    type: number
                    extended:
                    title: Extended
                    type: string
"""
getAssetsByOwner = {
    "name": "getAssetsByOwner",
    "description": "Accepts Solana publicKey address. Returns Metaplex NFTs owned by the address",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The publicKey address of the account"
            },
        },
        "required": ["address"],
    },
}

getAccountInfo = {
    "name": "getAccountInfo",
    "description": "Accepts Solana publicKey address. Returns balance and information about the account data",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The publicKey address of the account"
            },
        },
        "required": ["address"],
    },
}

getTokenAccounts = {
    "name": "getTokenAccounts",
    "description": "Returns the fungible and non-fungible tokens and amounts owned by the address. May show tokens not listed in getAssetsByOwner",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The publicKey address of the account"
            },
        },
        "required": ["address"],
    },      
}

getSignaturesForAddress = {
  "name": "query_signatures_for_address",
  "description": "Accepts Solana publicKey address. Returns the latest transaction signatures that involve that address",
  "parameters": {
    "type": "object",
    "properties": {
      "json": {
        "properties": {
          "address": {
            "type": "string"
          },
          "beforeSignature": {
            "type": "string"
          },
          "untilSignature": {
            "type": "string"
          }
        },
        "type": "object",
        "required": [
          "address"
        ],
        "title": "GetSignaturesForAddressRequest"
      }
    }
  }
}


functions = [getAssetsByOwner, getAccountInfo, getTokenAccounts]


In [3]:
messages = [{"role": "user", "content": "tell me the balance in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"}]
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "auto"
)
reply_content = completion.choices[0]
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
# print(funcs)
print("Message: {}".format(messages))
print("Reply Content: {}".format(reply_content))
print("funcs: {}".format(funcs))

Message: [{'role': 'user', 'content': 'tell me the balance in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}]
Reply Content: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"address\": \"8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625\"\n}",
    "name": "getAccountInfo"
  },
  "role": "assistant"
}
funcs: {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}


In [4]:
messages = [{"role": "user", "content": "tell me the NFTs in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"}]
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "auto"
)
reply_content = completion.choices[0]
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
# print(funcs)
print("Message: {}".format(messages))
print("Reply Content: {}".format(reply_content))
print("funcs: {}".format(funcs))

Message: [{'role': 'user', 'content': 'tell me the NFTs in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}]
Reply Content: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"address\": \"8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625\"\n}",
    "name": "getAssetsByOwner"
  },
  "role": "assistant"
}
funcs: {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}


In [5]:
messages = [{"role": "user", "content": "tell me all kinds of tokens present in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"}]
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "auto"
)
reply_content = completion.choices[0]
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
# print(funcs)
print("Message: {}".format(messages))
print("Reply Content: {}".format(reply_content))
print("funcs: {}".format(funcs))

Message: [{'role': 'user', 'content': 'tell me all kinds of tokens present in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}]
Reply Content: {
  "content": null,
  "function_call": {
    "arguments": "{\n\"address\": \"8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625\"\n}",
    "name": "getTokenAccounts"
  },
  "role": "assistant"
}
funcs: {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}


## Experiments to incorporate the path in the output

In [19]:

getAssetsByOwner = {
    "name": "getAssetsByOwner",
    "description": " Returns Metaplex NFTs owned by the address",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625kolksldfjssfsfsfs"
            },
            "path": {
              "type": "string",
              "description": "/api/handlers/getAssetsByOwner",
            },
            "location": {
              "type": "string",
              "description": "localHost",
            },
            "dadName": {
              "type": "string",
              "description": "ram",
            },
        },
        "required": ["address", "path", "location", "dadName"],
    },
}

getAccountInfo = {
    "name": "getAccountInfo",
    "description": "Accepts Solana publicKey address. Returns information the account data",
    "parameters": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The publicKey address of the account"
            },
        },
        "required": ["address"],
    },
}

functions = [getAssetsByOwner, getAccountInfo]


In [20]:
messages = [{"role": "user", "content": "tell me the NFTs in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625"}]
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions,
    function_call = "auto"
)
reply_content = completion.choices[0]
reply_content = completion.choices[0].message

funcs = reply_content.to_dict()['function_call']['arguments']
funcs = json.loads(funcs)
# print(funcs)
print("Message: {}".format(messages))
print("Reply Content: {}".format(reply_content))
print("funcs: {}".format(funcs))

Message: [{'role': 'user', 'content': 'tell me the NFTs in account 8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625'}]
Reply Content: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"address\": \"8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625\",\n  \"path\": \"/api/handlers/getAssetsByOwner\",\n  \"location\": \"localHost\",\n  \"dadName\": \"ram\"\n}",
    "name": "getAssetsByOwner"
  },
  "role": "assistant"
}
funcs: {'address': '8fbqVvpK3Dj7fdP2c8JJhtD7Zy3n9qtwAeGfbkgPu625', 'path': '/api/handlers/getAssetsByOwner', 'location': 'localHost', 'dadName': 'ram'}


## Conclusions:
- The endpoint is understanding the request format alright
- Need to process the OpenAPI spec into the input format for the function endpoint
- Need to process the endpoint output and make the API call. Preferably write an agent around it.
- So you can return the correct path from the function by two ways:
    - You make an external map and use it to check the path for the given endpoint
    - You provide the path in the description parameter and make it as required. Now you’ll have to benchmark this, that does this work.
    - The map seems better because we don't know how dependable is getting a fixed value out from description